In [1]:
using NamedColors
using Colors
using PooledElements
using TensorFlow
using Distributions
using Iterators

In [2]:
function get_raw_data() 
    local xs=String[]
    local ys=HSL[]
    for (x, y) in load_xkcd()
        push!(xs, x)
        push!(ys, convert(HSL, y))
    end
    
    xs, ys
end

const xs_raw, ys_raw = get_raw_data()

(String["grass green","lipstick","celery","chocolate","pine green","dark brown","twilight blue","purple red","light olive green","heliotrope"  …  "pale olive green","light bright green","pale peach","light burgundy","pastel purple","dusty lavender","periwinkle blue","brown","marine","light grass green"],ColorTypes.HSL[HSL{Float32}(98.3333,0.86747,0.32549),HSL{Float32}(342.632,0.805085,0.462745),HSL{Float32}(94.6154,0.962963,0.788235),HSL{Float32}(26.4407,0.936508,0.123529),HSL{Float32}(139.355,0.756098,0.160784),HSL{Float32}(31.2,0.925926,0.105882),HSL{Float32}(209.464,0.848485,0.258824),HSL{Float32}(332.368,0.987013,0.301961),HSL{Float32}(75.9184,0.429825,0.552941),HSL{Float32}(289.88,0.892473,0.635294)  …  HSL{Float32}(82.7586,0.491526,0.652941),HSL{Float32}(123.158,0.98844,0.660784),HSL{Float32}(40.9756,1.0,0.839216),HSL{Float32}(344.854,0.44206,0.456863),HSL{Float32}(266.526,1.0,0.813725),HSL{Float32}(306.316,0.186274,0.6),HSL{Float32}(234.444,0.931035,0.772549),HSL{Float32}(32.673

In [3]:
encode{I}(iter::I) = encode(eltype(I), iter)
encode{T}(::Type{T}, xs) = map(encode, xs)

function encode{S<:AbstractString}(::Type{S}, xs_raw) 
    tokens = Set{PooledString}()
    xs = PooledStringArray[]
    for x_raw in xs_raw
        x = PooledStringArray(encode(x_raw))
        push!(tokens, x...)
        push!(xs, x)
    end
    xs, tokens
end

encode(x_raw::AbstractString) = pstring.(split(x_raw))
encode{T<:PooledString}(::Type{T}, xs) = map(encode, xs) #Exception to the general rule for abstract strings
encode(ss::PooledString)=convert(Int32, ss.level)

encode(y::HSL) = Float32[sind(y.h), cosd(y.h), y.s, y.l]


encode (generic function with 7 methods)

In [4]:
const xs_tokens, tokens = encode(xs_raw)
const xs_ids = encode(xs_tokens)

949-element Array{Array{Int32,1},1}:
 Int32[1,2]    
 Int32[3]      
 Int32[4]      
 Int32[5]      
 Int32[6,2]    
 Int32[7,8]    
 Int32[9,10]   
 Int32[11,12]  
 Int32[13,14,2]
 Int32[15]     
 Int32[16,2]   
 Int32[17,18]  
 Int32[16,8]   
 ⋮             
 Int32[123,12] 
 Int32[407]    
 Int32[25,14,2]
 Int32[13,66,2]
 Int32[25,161] 
 Int32[13,268] 
 Int32[129,11] 
 Int32[44,147] 
 Int32[273,10] 
 Int32[8]      
 Int32[336]    
 Int32[13,1,2] 

In [5]:
const ys = encode(ys_raw)

949-element Array{Array{Float32,1},1}:
 Float32[0.989442,-0.144932,0.86747,0.32549]   
 Float32[-0.298515,0.954405,0.805085,0.462745] 
 Float32[0.996757,-0.0804665,0.962963,0.788235]
 Float32[0.445271,0.895396,0.936508,0.123529]  
 Float32[0.651373,-0.758758,0.756098,0.160784] 
 Float32[0.518027,0.855364,0.925926,0.105882]  
 Float32[-0.491881,-0.870662,0.848485,0.258824]
 Float32[-0.463785,0.885948,0.987013,0.301961] 
 Float32[0.96995,0.243304,0.429825,0.552941]   
 Float32[-0.94041,0.340043,0.892473,0.635294]  
 Float32[0.895712,0.444635,0.961538,0.203922]  
 Float32[0.909632,0.415415,0.488889,0.647059]  
 Float32[0.652637,0.757671,0.72973,0.217647]   
 ⋮                                             
 Float32[0.161058,0.986945,0.637306,0.378431]  
 Float32[0.655284,0.755383,0.440415,0.621569]  
 Float32[0.992024,0.12605,0.491526,0.652941]   
 Float32[0.837166,-0.546948,0.98844,0.660784]  
 Float32[0.655738,0.754989,1.0,0.839216]       
 Float32[-0.261273,0.965265,0.44206,0.456863]  
 

In [6]:
sess = Session(Graph())

Session(Ptr{Void} @0x00007fb10df62b90)

In [7]:
embedding_dim = 16
hidden_layer_size = 128
output_dim = 4
variable_scope("counting_model3", initializer=Normal(0, .1)) do
    global const Y_obs = placeholder(Float32)
    global const X_ids = placeholder(Int32)
    global const Ts = get_variable("TokenEmbeddings",  [length(tokens), embedding_dim], Float32)
    global const X_embs = nn.embedding_lookup(Ts,  X_ids);
    global const SoWe = reduce_sum(X_embs; reduction_indices=1, keep_dims=true); #Sum up however many embeddings we have
    
    global const W1 = get_variable("weights1", [embedding_dim, hidden_layer_size], Float32)
    global const B1 = get_variable("bias1", [hidden_layer_size,], Float32)    
    
    global const Wo = get_variable("weightsout", [hidden_layer_size, output_dim], Float32)
    global const Bo = get_variable("biasout", [output_dim,], Float32)    
    
    global const H = nn.sigmoid(SoWe*W1 + B1)
    global const Y = nn.tanh(H*Wo + Bo)
    global const Loss = reduce_sum((Y - Y_obs)^2)

end;

In [8]:
optimizer = train.GradientDescentOptimizer(0.025)
#optimizer = train.MomentumOptimizer(0.025f0, 0.005f0)
minimize_op = train.minimize(optimizer, Loss);

In [9]:
run(sess, initialize_all_variables())

0-element Array{Any,1}

In [84]:
#Lets do this as a batch
cur_loss, _, = run(sess,
                   vcat(Loss, minimize_op),
                   Dict(X_ids=>xs_ids, Y_obs=>ys))

LoadError: MethodError: Cannot `convert` an object of type Array{Float32,1} to an object of type Float32
This may have arisen from a call to the constructor Float32(...),
since type constructors fall back to convert methods.

In [ ]:
# Run training
#run(sess, initialize_all_variables())
recent_loss_total = 0.0
loss_window = 10000
for (ii, (x_id, y)) in enumerate(take(cycle(zip(xs_ids, ys)), 20_000))
    cur_loss, _, = run(sess,
                      vcat(Loss, minimize_op),
    Dict(X_ids=>x_id, Y_obs=>y))
    
    recent_loss_total += cur_loss
    if ii % loss_window == 0
        recent_loss = recent_loss_total/loss_window
        println(recent_loss)
        recent_loss_total =0.0
    end
end

In [35]:
function decode{T<:Real}(::Type{HSL}, y::AbstractVector{T})
    h = rad2deg(atan2(y[1], y[2]))
    s = y[3]
    l = y[4]
    
    HSL(h,s,l)
end

decode{T<:AbstractString}(::Type{T}, id::Integer, poolindex=PooledElements.__GLOBAL_POOL__.index) = poolindex[id]

decode (generic function with 3 methods)

In [37]:
decode(HSL, Float32[0.839931, 0.9263544, 0.767989, 0.449593])

In [38]:
function get_color(name)
    name_tokens = encode(name)
    x_id = encode(name_tokens)
    max_known_word_id = get_shape(Ts).dims |> first |> get
    filter!(id->id ≤ max_known_word_id, x_id)
    if length(x_id) < length(name_tokens)
        
        warn("Discarded: ", length(name_tokens) - length(x_id), " tokens, ", 
        "remaining: ", length(x_id))
    end
    
    
    y_coded = run(sess, vcat(Y), Dict(X_ids=>x_id)) |> first |> vec
    @show y_coded
    col = decode(HSL, y_coded)
    @show convert(RGB, col)
    col
end

get_color (generic function with 1 method)

In [56]:
get_color("darkish dark light light tan")

y_coded = Float32[0.718403,0.513588,0.19138,0.879054]
convert(RGB,col) = RGB{Float32}(0.90220034f0,0.8979097f0,0.85590684f0)


In [57]:
get_color("grass green")

y_coded = Float32[0.957458,-0.098743,0.790666,0.567506]
convert(RGB,col) = RGB{Float32}(0.50038916f0,0.9094641f0,0.22554708f0)


In [58]:
get_color("pale blue green")

y_coded = Float32[0.673799,-0.879389,0.922094,0.848319]
convert(RGB,col) = RGB{Float32}(0.70845395f0,0.9881831f0,0.81353974f0)


In [59]:
get_color("grey")

y_coded = Float32[0.209482,0.376455,0.280165,0.666978]
convert(RGB,col) = RGB{Float32}(0.76027894f0,0.66416025f0,0.57367635f0)


In [73]:
get_color("black")

y_coded = Float32[0.144143,0.951593,0.100851,0.256596]
convert(RGB,col) = RGB{Float32}(0.28247347f0,0.23814756f0,0.23071763f0)


In [61]:
get_color("ivory")

y_coded = Float32[0.737909,0.905667,0.883901,0.798466]
convert(RGB,col) = RGB{Float32}(0.976602f0,0.8529282f0,0.6203293f0)


In [62]:
get_color("reddish red")

y_coded = Float32[-0.0368657,0.999585,0.936841,0.540289]
convert(RGB,col) = RGB{Float32}(0.9709653f0,0.10961288f0,0.1399349f0)


In [63]:
get_color("red")

y_coded = Float32[0.0970419,0.993028,0.945126,0.596034]
convert(RGB,col) = RGB{Float32}(0.9778329f0,0.28526717f0,0.21423471f0)


In [64]:
get_color("reddish")

y_coded = Float32[0.0309445,0.97747,0.755551,0.589044]
convert(RGB,col) = RGB{Float32}(0.8995421f0,0.2973122f0,0.27854514f0)


In [65]:
get_color("blue red")

y_coded = Float32[-0.526487,0.899951,0.966353,0.597983]
convert(RGB,col) = RGB{Float32}(0.9864732f0,0.2094934f0,0.6022358f0)


In [66]:
get_color("blue green")

y_coded = Float32[0.450142,-0.865818,0.864836,0.59704]
convert(RGB,col) = RGB{Float32}(0.24854547f0,0.9455343f0,0.6264285f0)


In [67]:
get_color("blue")

y_coded = Float32[-0.438923,-0.57753,0.862064,0.648539]
convert(RGB,col) = RGB{Float32}(0.34555656f0,0.5754707f0,0.95152074f0)


In [68]:
get_color("blue blue")

y_coded = Float32[-0.77687,-0.957892,0.911028,0.652946]
convert(RGB,col) = RGB{Float32}(0.33676916f0,0.5576415f0,0.9691219f0)


In [69]:
get_color("bluish blue")

y_coded = Float32[-0.853077,-0.896532,0.751147,0.626851]
convert(RGB,col) = RGB{Float32}(0.34656036f0,0.4999983f0,0.90714073f0)


In [70]:
get_color("bluish")

y_coded = Float32[-0.589034,-0.154932,0.622888,0.623845]
convert(RGB,col) = RGB{Float32}(0.5087505f0,0.38954198f0,0.85814714f0)


In [71]:
get_color("bluish black")

y_coded = Float32[-0.619058,0.754579,-0.218318,0.224795]
convert(RGB,col) = RGB{Float32}(0.17571782f0,0.27387148f0,0.2094737f0)


In [72]:
get_color("dark black")

y_coded = Float32[-0.0643716,0.906893,-0.111987,0.0468052]
convert(RGB,col) = RGB{Float32}(0.041563615f0,0.052046813f0,0.05133744f0)
